### Développement d’un pipeline de calcul du TMB

#### Projet 4BiM, 2021

#### Auteurs : Marie Casimir, Loup Petitjean et Nicolas Mendiboure
#### Encadrantes Innate-Pharma : Sabrina Carpentier et Luciana Bastista
#### Encadrante INSA : Maïwenn Pineau

### Installation et import des modules

In [1]:
import io
import copy
import pandas as pd
import numpy as np
from collections import Counter

## Importation de vcf sample1 :

In [2]:
def check_extension(path):
    #Vérifier que l'on donne bien un fichier dont l'extension est .vcf :
    
    split = path.split(".")
    format_name = len(split) -1 

    if (split[format_name].lower() != "vcf"):
        print("Votre fichier n'est pas au bon format, format attendu : vcf")
        return(False)
    else :
        print("Succès : extension vcf détectée")
        return(True)

In [3]:
def check_format(file):
    # On regarde que la première ligne soit bien de la forme ##format=VCFv4.x
    with open(file, 'r') as f:
        line = f.readline()
    
    if (line.find("VCF") != -1): #Si on trouve 'VCF' dans line 
        return (True)
    else:
        return(False)

In [4]:
def check_missing_data(df):
    """
    Code pour les returns:
    -1 : False, le fichier n'est pas bon on le rejette ;
    0 : Le fichier est à la limite de l'acceptable (quelques NaN) 
    et peut subir une modification pour traiter les NaN
    1  True, le fichier est validé 
    """
    #Vérifier  qu'il ne manque pas une colonne dans le ficher :
    columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT']
    for col in columns:
        if (col not in df.columns):
            print("Ce fichier vcf est incomplet")
            print("Il manque la colonne {}".format(col))
            return (-1)
        
    #Vérifier que chaque ligne est bien complète (pas de NaN): 
    NaN_col = df.isna().sum(axis=0)
    NaN_cutoff = 3 #nombre de NaN admissibles par fichier vcf, au dèla fichier rejetté.
    
    if (sum(NaN_col) !=0) : #s'il y a des NaN
        if (sum(NaN_col) <= NaN_cutoff) :
            return(0)
            
        else:
            print("Votre fichier un nombre de données manquantes trop élevé.")
            print("Veuillez fournir un vcf de meilleur qualité.")
            return (-1)
    else:
        print("Contrôle des NaN : Ok")
        return(1)

In [5]:
def check_filter(df, minimum_PASS = 0.85):
    
    """ 
    minimum_PASS: minimal number of lines where filter == from PASS : 85% by default
    return False if the vcf quality is too low, if there a enough PASS, the dataframe is kept ,
    and we will remove all the lines which do not have a "PASS" argument in the FILTER column.
    """
    
    new_df = copy.deepcopy(df)
    FILTER = df["FILTER"].values
    count = Counter(FILTER)
    
    if (count['PASS'] < minimum_PASS*len(df)):
        print("Votre fichier est de mauvaise qualité, veuilliez en charger un nouveau")
        return (False)
    
    elif(len(count) == 1):
        print("Contrôle des filtres : RAS")
        return (df)
    
    elif (count['PASS'] >= minimum_PASS*len(df)) and (len(count) >1):
        indexes = list(np.concatenate(np.argwhere(FILTER != "PASS")))
        for i in indexes :
            new_df = new_df.drop(labels=i, axis=0)
            
        print("Contrôle des filtres : OK")
        return(new_df)

In [6]:
def drop_NaN_rows(df):
    new_df = copy.deepcopy(df) # Pour ne pas ecraser notre df initial 
    NaN_line = df.isna().sum(axis=1)
    indexes = []
    for i, line in enumerate(NaN_line.values):
                if (line != 0):
                    indexes.append(i)
    for idx in indexes :
                new_df = df.drop([idx], axis = 0)
            
    print("Suppression des NaN : Ok")      
    return (new_df)

In [7]:
def quality_control(df):
    
    miss = check_missing_data(df)
    print(miss)
    
    if (miss == -1):
        return (False)
    
    elif (miss == 1):
        df2 = check_filter(df)
        
        if (type(df2) == bool):
            if(df2 == False):
                return (None)
        else:
            return(df2)
    
    elif (miss == 0):
        df2 = drop_NaN_rows(df) #remove NaN rows
        df3 = check_filter(df2) #check the number of PASS filters
        
        if (type(df3) == bool):
            if(df3 == False):
                return (None)
        else:
            return(df3)

In [8]:
def read_vcf(path, QC = True):
    if (check_extension(path) == True) and (check_format(path) == True) :
        with open(path, 'r') as f:
            lines = [l for l in f if not l.startswith('##')]
        
        vcf_df = pd.read_csv( io.StringIO(''.join(lines)), dtype={'#CHROM': str, 'POS': int, 'ID': str, 
                                                               'REF': str, 'ALT': str,'QUAL': str, 
                                                               'FILTER': str, 'INFO': str, 'FORMAT': str}, 
                         sep='\t').rename(columns={'#CHROM': 'CHROM'})
        if (QC == True):
            vcf_df2 = quality_control(vcf_df) #On passe notre vcf en contrôle qualité
            if(type(vcf_df2) != bool):
                vcf_df2.index = range(0, len(vcf_df2),1) #car les indexes ne sont plus bons à cause du df.drop...
            print("Contrôle qualité du VCF : Ok")
            return(vcf_df2)
        else:
            print("Contrôle qualité du VCF : Non fait")
            return(vcf_df)

## Verification des inputs :

In [10]:
#Test avec un fichier où il manque une colonne :

test_colonne = read_vcf("vcf_files/sample1/vcf_incomplet_manque_1_colonne.vcf", QC=True)
print(test_colonne)

Succès : extension vcf détectée
Ce fichier vcf est incomplet
Il manque la colonne POS
-1
Contrôle qualité du VCF : Ok
False


In [11]:
#Test avec un fichier où in y a des NaN sur une ligne :

test_ligne = read_vcf("vcf_files/sample1/vcf_ligne_incomplete.vcf")
test_ligne.tail(4)

Succès : extension vcf détectée
0
Suppression des NaN : Ok
Contrôle des filtres : OK
Contrôle qualité du VCF : Ok


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Sample1-PBMC_1
5,1,762601,7,T,C,221.84,PASS,AC=2;AF=1;AN=2;DB;DP=6;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,6:6:18:250,18,0"
6,1,762632,8,T,A,57.74,PASS,AC=2;AF=1;AN=2;DB;DP=2;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,2:2:6:85,6,0"
7,1,777122,9,A,T,100.03,PASS,AC=2;AF=1;AN=2;DB;DP=4;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,4:4:12:128,12,0"
8,1,783304,10,T,C,324.78,PASS,AC=2;AF=1;AN=2;DB;DP=8;FS=0;MLEAC=2;MLEAF=1;MQ...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,8:8:24:353,24,0"


In [12]:
sum(test_ligne.isna().sum(axis=1)) #On a plus de NaN !

0

In [13]:
sample1_tumor = read_vcf("./vcf_files/sample1/Sample1_tumor_dna.vcf", QC=True)

Succès : extension vcf détectée
Contrôle des NaN : Ok
1
Contrôle des filtres : OK
Contrôle qualité du VCF : Ok


In [14]:
sample1_normal = read_vcf("./vcf_files/sample1/Sample1-PBMC_normal_dna.vcf", QC=True)

Succès : extension vcf détectée
Contrôle des NaN : Ok
1
Contrôle des filtres : OK
Contrôle qualité du VCF : Ok


In [15]:
sample1_somatic = read_vcf("./vcf_files/sample1/Sample1_somatic_dna.vcf", QC=True)

Succès : extension vcf détectée
Contrôle des NaN : Ok
1
Contrôle des filtres : OK
Contrôle qualité du VCF : Ok


In [16]:
print(len(sample1_normal), len(sample1_tumor), len(sample1_somatic))

148754 159734 244


In [17]:
sample1_tumor.head()
#sample1_somatic.head()
#sample1_normal.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Sample1
0,1,752721,1,A,G,729.77,PASS,AC=2;AF=1;AN=2;DB;DP=21;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,21:21:63:758,63,0"
1,1,761957,3,A,AT,1252.73,PASS,AC=2;AF=1;AN=2;DB;DP=31;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,31:31:96:1290,96,0"
2,1,762273,4,G,A,11593.8,PASS,AC=2;AF=1;AN=2;DB;DP=357;FS=0;MLEAC=2;MLEAF=1;...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,357:357:99:11622,1071,0"
3,1,762589,5,G,C,1841.77,PASS,AC=2;AF=1;AN=2;DB;DP=42;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,42:42:99:1870,126,0"
4,1,762592,6,C,G,1841.77,PASS,AC=2;AF=1;AN=2;DB;DP=41;FS=0;MLEAC=2;MLEAF=1;M...,GT:GATK:AD:DP:GQ:PL,"1/1:1/1:0,41:41:99:1870,126,0"


## Creation de fichiers 'lite'  (chr1)

In [18]:
sample1_tumor_chr1 = sample1_tumor.loc[sample1_tumor["CHROM"] == '1']
sample1_normal_chr1 = sample1_normal.loc[sample1_normal["CHROM"] == '1']
sample1_somatic_chr1 = sample1_somatic.loc[sample1_somatic["CHROM"] == '1']

In [19]:
print(len(sample1_normal_chr1), len(sample1_tumor_chr1), len(sample1_somatic_chr1))

14021 15052 23


## Différence entre tumor et normal :

In [126]:
def compare(df_tumor, df_normal):
    """
    On regarde si une mutation présente dans le fichier vcf tumoral 
    est également présente dans le fichier vcf sain (dit normal). 
    Dans le cas échéant on ne compte pas cette mutation comme étant une mutation propre à la tumeur,
    car elle est localisée à d'autres endroits dans l'organisme (simple SNP par exemple).
    Dans le cas contraire on peut considèrer cette mutation comme étant somatic est propre à la tumeur.
    """
    CHROMS = np.unique(df_tumor['CHROM'].values)
    indexes = []
    
    for _chr_ in CHROMS:
        df_t = df_tumor.loc[df_tumor["CHROM"] == _chr_]
        df_n = df_normal.loc[df_normal["CHROM"] == _chr_]
    
        #df to np
        POS_tumor = df_t['POS'].values
        POS_normal = df_n['POS'].values

        for muta in df_tumor.index:
            NB_ALT_tumor = len(list(df_t['ALT'][muta]))
            START = POS_tumor[muta]
            END = START + len(list(df_t['ALT'][muta]))

            #On identifie une mutation par ses positions de début et de fin (start et end)
            if (START in list(POS_normal)): #même debut ?
                # !!une meme position peut sur des chr differents!! 
                index = int(np.argwhere(POS_normal == START))
                if (END == POS_normal[index] + len(list(df_n['ALT'][index]) ) ): #même fin ?
                    pass # non somatique
                else:
                    indexes.append(df_t['ID'][muta])
    return (indexes)

In [127]:
indexes = compare(sample1_tumor_chr1, sample1_normal_chr1)

In [128]:
len(indexes)

80

In [129]:
def create_somatic(tumor_path, somatic_path, indexes):
    """
    Cette fonction permet d'écrire dans un fichier toutes les mutations présentes dans le tissus tumoral,
    et absent dans le tissus (ou échantillon) dit normal. Cette comparaison est faite avec la fonction compare
    ci dessus. 
    """
    
    headers = []
    lines = []

    with open(tumor_path, "r") as f_in:
        for line in f_in:
            if line.startswith('#'):
                headers.append(line)
            elif not line.startswith('#'):
                lines.append(line)

    with open(somatic_path, "w") as f_out:
        for i in range(len(headers)):
                f_out.write(headers[i])
        for j in range(len(lines)-1):
            if (lines[j].split()[2] in indexes):
                f_out.write(lines[j])
                
    return(0)

In [131]:
create_somatic("./vcf_files/sample1/Sample1_tumor_dna_chr1.vcf", 
               "./vcf_files/sample1/Sample1_our_somatic_chr1.vcf", indexes)

0

## Calcul du TMB :

In [20]:
# Savoir si une mutation est synonyme ou pas :

def is_synonymous():
    pass

In [44]:
def TMB_without_somatic(df_tumor, df_normal, exome_length = 1):
    TMB= len(df_tumor) - len(compare(df_tumor, df_normal))
    return(TMB/exome_length)

In [30]:
def TMB_with_somatic(df_somatic, exome_length = 1):
    """
    Ici on calcul notre TMB directement à partir d'un fichier vcf somatic,
    il y a donc moins d'étapes. Le calcul revient à prendre 
    la longueur du nombre de mutations somatiques totales
    """
    
    TMB = len(df_somatic['ALT'].values)
    return (TMB/exome_length)

In [31]:
## Bcp trop long à calculer... (pas fini après plus d'1h)

#TMB_without_somatic(sample1_tumor, sample1_normal)

In [45]:
TMB_without_somatic(sample1_tumor_chr1, sample1_normal_chr1)

1660.0

In [34]:
TMB_with_somatic(sample1_somatic_chr1)

23.0

### Test des librairies pyvcf et cyvcf :


In [ ]:
#pip install cyvcf2
#pip install pyvcf
#pip install hgvs

In [ ]:
import cyvcf2
import vcf

In [ ]:
## Importation d'un VCF avec la librairie cyvcf2 :
## Si les colonnes INFO et FORMAT sont de la forme : AC=40;AF=0.1;AN=2;BQ=36;DP=413;FA=0.1;INDEL=0;MC=C>T

def read_cyvcf(vcf):

    #Général :

    CHROM = []
    POS = []
    REF = []
    ALT = []
    QUAL = []
    FILTER = []

    # Détails de la section INFO du VCF :

    AN = [] #Total number of alleles in called genotypes
    AC = [] #Allele count in genotypes, for each ALT allele, in the same order as listed
    AF = [] #Allele Frequency in primary data, for each ALT allele, in the same order as listed
    BQ = [] #RMS base quality
    SB = [] #Strand bias
    FA = [] #Overall fraction of reads supporting ALT
    MC = [] #Modification base changes at this position
    MT = [] #Modification types at this position
    NS = [] #Number of Samples With Data
    DP = [] #Total Depth across samples
    VT = [] #Variant type, can be SNP, INS or DEL
    SS = [] #Variant status relative to non-adjacent Normal,0=wildtype,1=germline,2=somatic,3=LOH,4=post-transcriptional modification,5=unknown
    ORIGIN = [] #Where the call originated from, the tumor DNA, RNA, or both
    SOMATIC = [] #Indicates if record is a somatic mutation
    INDEL = [] #Number of indels for all samples
    START = [] #Number of reads starting at this position across all samples
    STOP = [] #Number of reads stopping at this position across all samples



    for record in cyvcf2.VCF(vcf):
        CHROM.append(record.CHROM)
        POS.append(record.POS)
        REF.append(record.REF)
        ALT.append(record.ALT)
        QUAL.append(record.QUAL)
        FILTER.append(record.FILTER)

        #record.INFO est un objet de type cyvcf, pour extraire les données il faut utiliser .get()
        AN.append(record.INFO.get("AN"))
        AC.append(record.INFO.get("AC"))
        AF.append(record.INFO.get("AF"))
        BQ.append(record.INFO.get("BQ"))
        SB.append(record.INFO.get("SB"))
        FA.append(record.INFO.get("FA"))
        MC.append(record.INFO.get("MC"))
        MT.append(record.INFO.get("MT"))
        NS.append(record.INFO.get("NS"))
        DP.append(record.INFO.get("DP"))
        VT.append(record.INFO.get("VT"))
        SS.append(record.INFO.get("SS"))
        ORIGIN.append(record.INFO.get("ORIGIN"))
        SOMATIC.append(record.INFO.get("SOMATIC"))
        INDEL.append(record.INFO.get("INDEL"))
        START.append(record.INFO.get("START"))
        STOP.append(record.INFO.get("STOP"))

    df_VCF = pd.DataFrame(list(zip(CHROM, POS, REF, ALT, QUAL, FILTER)), 
                      columns=["CHROM", "POS", "REF", "ALT", "QUAL", "FILTER"])

    df_VCF[ ["AN", "AC", "AF", "BQ", "SB", "FA", "MC", 
         "MT", "NS", "DP", "VT", "SS", "ORIGIN", 
         "SOMATIC", "INDEL", "START", "STOP"]] = pd.DataFrame(list(zip (AN, AC, AF, BQ, SB, FA, MC,
                                                                        MT, NS, DP, VT, SS, ORIGIN, 
                                                                        SOMATIC, INDEL, START, STOP)))
    return (df_VCF)

In [ ]:
#radia = read_cyvcf("./vcf_files/test_radia.vcf")
#radia_alt = radia['ALT'].values
#print(radia_alt[0:6])

In [ ]:
#radia.head()

In [ ]:
#basic_radia_TMB = len(radia_alt)
#print(basic_radia_TMB)

### Fichier mutect.vcf

In [ ]:
# Avec la librairie pyvcf :

def read_pyvcf(file):
    reader = vcf.Reader(open(file))
    df = pd.DataFrame([vars(r) for r in reader])
    out = df.merge(pd.DataFrame(df.INFO.tolist()),
                   left_index=True, right_index=True)
    return out

In [ ]:
#mutect = read_pyvcf("./vcf_files/test_mutect.vcf")
#mutect_alt = mutect['ALT'].values
#print(mutect_alt[:6])

In [ ]:
#basic_mutect_TMB = len(mutect_alt)
#print(basic_mutect_TMB)

In [ ]:
#mutect.head()

In [ ]:
#muse = read_pyvcf("./vcf_files/test_muse.vcf")
#muse.head()

In [ ]:
#merged = read_pyvcf("./vcf_files/test_merged.vcf")
#merged.head()

### Calcul du TMB en comparant un tissu sain et un tissu malade 

Comme on n'arrive pas à trouver une paire de jeu de données correctes, on va la fabriquer à partir du fichier mutect.vcf : on va enlever les 2/3 des lignes du fichier, et on considèrera que c'est notre tissu sain de référence. 

"In order to filter out germline variants, the ideal situation would be to
sequence a matched non-tumor sample from each patient."

#### Fonction simple

On considère ici que si la mutation est présente dans le tissu sain, c'est une mutation synonyme.

In [ ]:
#mutect_tumor = read_pyvcf("./vcf_files/test_mutect.vcf")
#mutect_sain = read_pyvcf("./vcf_files/test_mutect_sain.vcf")

### Cas où l'on n'a pas de tissu sain

On doit comparer nos mutations aux mutations d'une base de donnée pour voir si ces mutations sont dues au polymorphisme ou non.


On utilise la libraireie HGVS de python pour comparer nos variations à des variations courantes, contenues dans les BDD : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6282708/ 

Tuto ici : https://github.com/biocommons/hgvs


In [ ]:
#import hgvs.dataproviders.uta